# Segmenting and Clustering Neighborhoods in Toronto

*Afrioni Roma Rio*

In [1]:
# import the libraries
import numpy as np 
import pandas as pd 
import folium

In [2]:
# scraping the wikipedia page
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df_url = pd.read_html(url)

In [32]:
# the table found in element [0]
df_canada = df_url[0].copy()
df_canada.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [33]:
#ignore borough == not assigned
df_canada.drop(df_canada.loc[df_canada['Borough'] == 'Not assigned'].index, inplace = True)

#reset index 
df_canada.reset_index(drop = True, inplace=True)
df_canada.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [39]:
# check for data shape
df_canada.shape
print(f'It consist of {df_canada.shape[0]} rows and {df_canada.shape[1]} columns')

It consist of 103 rows and 3 columns


In [40]:
# get the geospatial data
df_geo = pd.read_csv('http://cocl.us/Geospatial_data')
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [43]:
# get the lat and long by postal code
geo_lat_lang = df_canada[df_canada['Postal Code'].isin(df_geo['Postal Code'].tolist())] 

In [47]:
# merge the data frames
df_geo_merge = pd.merge(geo_lat_lang, df_geo, 'inner', on= 'Postal Code')
df_geo_merge.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [54]:
# create map
map_toronto = folium.Map(location=[43.651070, -79.347015], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df_geo_merge['Latitude'], df_geo_merge['Longitude'], df_geo_merge['Borough'], df_geo_merge['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto